In [1]:
import pandas as pd
import numpy as np
import os
import uuid
from tkinter import filedialog
from tkinter import *
from typing import List

from scipy.stats import ks_2samp

pd.set_option('display.max_rows', None)

import config

DROP_IF = ["DO NOT FILL IN", "DO NOT F"]
PATH = r"C:\Users\MB2705851\OneDrive - Surbana Jurong Private Limited\Manuals & Guidelines\Traffic\Manual count import templates"

def getfiles(path: str) -> List[str]:
    print("COLLECTING FILES......")
    src = []
    for root, dirs, files in os.walk(path):
        for name in files:
            if name.endswith(".xlsx"):
                p = os.path.join(root, name)
                src.append(p)
    src = list(set(src))
    return src

In [2]:

for file in getfiles(PATH):

    xls = pd.ExcelFile(file)

    header_out_df = pd.DataFrame()
    data_out_df = pd.DataFrame()

    for sheet in xls.sheet_names:
        df = pd.read_excel(file, sheet_name=sheet, header=None)

        #  df.LOC[ROW, COLUMN] -- REMEMBER THIS
        header = {
            "header_id": [str(uuid.uuid4())],
            "counted_by": ["TCS Trust"],
            "tc_station_name": [str(df.loc[4, 8])+str(df.loc[5, 8])],
            "count_type_id": 3,
            "count_date_start": [df.loc[2, 1]],
            "count_weather": [df.loc[1, 2]],
            "h_station_date": [str(df.loc[4, 8])+str(df.loc[5, 8]) + "_" + str(df.loc[2, 1])],
            "growth_rate_use": [str("y")],
            "count_interval": [60],
            "latitude": [df.loc[14, 8]],
            "longitude": [df.loc[15, 8]],
            "kilometer_dist": [df.loc[8, 8]],
            "road_link": [df.loc[6, 8]],
            "type_of_count": [df.loc[13, 8]],
            "description": ["Between " + str(df.loc[9, 8]) + " and " + str(df.loc[10, 8])],
            "no_of_hours": [df.loc[24, 8]],
            "no_days": [df.loc[25, 8]]
        }
        header_temp = pd.DataFrame(header)
        header_out_df = header_out_df.append(header_temp)

        data = df.loc[4:29, 0:5]
        data = data[(data[0] != "Subtotal A") & (data[0] != "Subtotal B")]
        data = data.dropna(thresh=5)
        data.rename(
            columns={
                0: "count_hour",
                1: "light",
                2: "heavy",
                3: "bus",
                4: "taxi",
                5: "total",
            },
            inplace=True,
        )
        data["count_hour"] = data["count_hour"].str[:2]
        data["header_id"] = header_temp.loc[0, "header_id"]

        data["count_hour"] = pd.to_datetime(
            data["count_hour"].str[:8], format="%H"
        ).dt.time

        hour = data["count_hour"].astype(str)
        data["count_time"] = header_temp.loc[0, "count_date_start"]
        data["count_time"] = pd.to_datetime(
            data["count_time"], format="%y/%m/%d"
        ) + pd.to_timedelta(hour)

        data["header_date"] = header_temp.loc[0, "count_date_start"]
        data_out_df = data_out_df.append(data)


COLLECTING FILES......


In [24]:
# def check_if_cumulative_count(df)
a = data_out_df['total']
normalized_df = (a-a.min())/(a.max()-a.min())
if (normalized_df.head(1).all() == 0.0) & (normalized_df.tail(1).all() == 1):
    print(True)
else:
    print(False)

s = pd.DataFrame(config.KS_SAMPLE)
normalized_sample = (s-s.min())/(s.max()-s.min())
normalized_sample = np.array(normalized_sample[0])


# ks_2samp(normalized_sample, normalized_df)

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [4]:
# a = np.array(data_out_df['total'])
# a2 = np.linalg.norm(a)
# a = a/a2

# s3 = np.array(config.KS_SAMPLE2)
# ss3 = np.linalg.norm(s3)
# s3 = s3/ss3

# samp = np.array(config.KS_SAMPLE)
# samp2 = np.linalg.norm(samp)
# samp = samp/samp2
# print(a)
# print(samp)
# ks_2samp(s3, a)